In [1]:
from dswe import TempGP
import numpy as np
import pandas as pd

### Load dataset

In [2]:
df = pd.read_csv('../../Inland_Offshore_Wind_Farm_Dataset1/Inland Wind Farm Dataset1(WT1).csv')

In [3]:
df.head()

,Sequence No.,V,D,air density,I,S_b,y (% relative to rated power)
0,1,7.96,138.9,1.140224,0.090452,0.266512,39.315152
1,2,8.19,140.6,1.140522,0.083028,0.286167,45.745455
2,3,7.20,139.3,1.140771,0.098611,0.339321,38.212121
3,4,6.81,137.4,1.141186,0.101322,0.375815,28.218182
4,5,5.09,137.5,1.141464,0.165029,0.303472,12.721212


In [4]:
df.shape

(47542, 7)

### Feature selection and train-test split

In [5]:
X_train = df.iloc[:1000, 1:6].values
y_train = df.iloc[:1000, 6].values
T_train = df.iloc[:1000, 0].values

In [6]:
X_test = df.iloc[1000:1100, 1:6].values
y_test = df.iloc[1000:1100, 6].values
T_test = df.iloc[1000:1100, 0].values

In [7]:
X_update = df.iloc[1100:1500, 1:6].values
y_update = df.iloc[1100:1500, 6].values
T_update = df.iloc[1100:1500, 0].values

### Fit the TempGP model

In [8]:
tempGP_model = TempGP(opt_method='L-BFGS-B')

In [9]:
tempGP_model.fit(X_train, y_train, T_train)

In [10]:
tempGP_model.optim_result

{'estimated_params': {'theta': array([ 26.6168029 , 124.17708876,  49.49534424,  49.38179841,
          49.63940411]),
  'sigma_f': 38.0364389269656,
  'sigma_n': 32.87725705089644,
  'beta': 35.13901869370132},
 'obj_val': None,
 'grad_val': None}

### Prediction on the test set

In [11]:
prediction = tempGP_model.predict(X_test, T_test)
prediction

/opt/homebrew/Caskroom/miniforge/base/envs/condapy/lib/python3.9/site-packages/dswe/_tempGP_subroutine.py:148: UserWarning: While computing g(t), variance of the training residuals is numerically zero for time index: 1007
Using mean of the response as the prediction.
  warnings.warn(msg)


array([ 2.22075166, -2.18510218, -5.48026168, -2.0770907 , -6.40532669,
        3.46447964, -0.45221518, 30.28170117, 30.93966582, 29.35983662,
       29.90826361, 24.71214091, 22.52025708, 18.94996753, 22.52411445,
       22.5741802 , 24.17945606, 23.53091573, 23.04868467, 23.18472132,
       31.02742953, 25.80116975, 21.00754436, 11.63050986,  4.83330261,
       20.68786152, 14.56270156,  8.31251311,  9.7594407 , 19.71672193,
        5.12120944,  4.3429477 , -0.5270037 ,  2.18863687,  4.86718485,
       11.48727363, 12.8007834 , 12.02581097,  9.70499355, 12.31826052,
        8.654749  , 10.63928273,  6.65671207,  5.19477441,  5.76782579,
        8.18104762,  1.69498021, -1.58414985,  0.53681738, -0.16388618,
        1.90894395, -2.28343824,  5.31397131,  7.92570039,  7.89162916,
        9.00623492,  9.14103367,  8.58542143,  5.94073444,  3.89405081,
       -2.55957566, 17.27408473, 23.67214852, 26.39701848, 17.26339141,
       18.53446031, 28.01636826, 22.37501688,  4.77575834, 12.73

In [12]:
np.abs(prediction - y_test).mean()

6.108524153475761

### Update the model on new data

In [13]:
tempGP_model.update(X_update, y_update, T_update, replace=True, update_model_F=False)

### Prediction on new test data

In [14]:
X_test_new = df.iloc[1500:1600, 1:6].values
y_test_new = df.iloc[1500:1600, 6].values
T_test_new = df.iloc[1500:1600, 0].values

In [15]:
new_prediction = tempGP_model.predict(X_test_new, T_test_new)
new_prediction

/opt/homebrew/Caskroom/miniforge/base/envs/condapy/lib/python3.9/site-packages/dswe/_tempGP_subroutine.py:148: UserWarning: While computing g(t), variance of the training residuals is numerically zero for time index: 1507
Using mean of the response as the prediction.
  warnings.warn(msg)


array([ 4.27314612,  0.06025814,  3.32340725, -0.3552116 ,  5.52679607,
        5.64425221,  5.09055259,  7.43321816, 11.84683717, 15.71963252,
       18.41283526, 13.78274977, 18.10817046, 24.72278406, 22.22189604,
       22.69529134, 20.94591204, 25.15764203, 27.68447124, 23.63309948,
       23.55622882, 24.89933873, 22.2874042 , 15.80950508,  8.52203422,
        3.40111277,  8.92000966, 11.36473641,  7.20889116,  6.75912481,
       11.62869159,  7.93986719, 16.91226191, 17.619587  , 10.44029938,
        8.54744724, 11.16978547,  9.82891698, 11.94825633, 16.15711381,
        0.04700495,  0.66415117, -0.20896818, -1.71183502,  9.75891695,
        7.51640561,  7.87341682,  1.80858402,  1.27079229,  2.11636553,
        0.90894411,  0.8905617 , -2.16871673,  0.5835259 ,  2.95374106,
        5.12328755, 14.8145542 , 29.63235799, 36.54421071, 29.37896494,
       24.60922375, 26.01746055, 22.31815324, 17.18025417, 15.82898384,
       19.66402781, 16.52898027, 15.09641617, 14.63822176, 15.55

In [16]:
np.abs(new_prediction - y_test_new).mean()

5.321669121633244